#**Llama 2**

[Llama 2 13B-chat](https://huggingface.co/meta-llama/Llama-2-13b-chat)

The Hugging Face community provides quantized models, which allow us to efficiently and effectively utilize the model on the T4 GPU. It is important to consult reliable sources before using any model.

There are several variations available, but the ones that interest us are based on the GGLM library.

We can see the different variations that Llama-2-13B-GGML has [here](https://huggingface.co/models?search=llama%202%20ggml).



In this case, we will use the model called [Llama-2-13B-chat-GGML](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML).

#**Step 1: Install All the Required Packages**

In [ ]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4
!pip install transformers

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.6 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.5/819.5 kB 9.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 9.4 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.1/26.1 MB 55.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 30.7 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 8.1 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-d1fm0zv5/overlay/local/bin
    changing mode of /tmp/pip-build-env-d1fm0zv5/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-d1fm0zv5/overlay/lo

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

#**Step 2: Import All the Required Libraries**

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
import csv
import pandas as pd
import re
from transformers import AutoTokenizer
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#**Step 3: Download the Model**

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

llama-2-13b-chat.ggmlv3.q5_1.bin:   0%|          | 0.00/9.76G [00:00<?, ?B/s]

#**Step 4: Loading the Model**

In [ ]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=2048
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

#**Step 5: IMDB Classification Task**

In [ ]:
dataset_imdb = pd.read_csv("/content/imdb_test.csv")

In [ ]:
dataset_imdb

,text,label
0,I love sci-fi and am willing to put up with a ...,0
1,"Worth the entertainment value of a rental, esp...",0
2,its a totally average film with a few semi-alr...,0
3,STAR RATING: ***** Saturday Night **** Friday ...,0
4,"First off let me say, If you haven't enjoyed a...",0
...,...,...
24995,Just got around to seeing Monster Man yesterda...,1
24996,I got this as part of a competition prize. I w...,1
24997,I got Monster Man in a box set of three films ...,1
24998,"Five minutes in, i started to feel how naff th...",1


In [ ]:
df=dataset_imdb.sample(n=1000, random_state=1).reset_index()

In [ ]:
df

,index,text,label
0,21492,"Man, is it great just to see Young and The Res...",1
1,9488,Seems like a pretty innocent choice at first- ...,0
2,16933,I was into the movie right away. I've seen the...,1
3,12604,Anyone who appreciates fine acting and ringing...,1
4,8222,High school female track star dies of a blood ...,0
...,...,...,...
995,15877,This was the one movie to see about the Civi W...,1
996,4314,And the worst part is that it could have been ...,0
997,23081,The comments of the previous user are harsh in...,1
998,24354,The Cure is an outstanding real-life drama tha...,1


In [ ]:
df=df[1:501]

In [ ]:
df.to_csv('IMDB-test-sampled.csv')

#**Step 6: Generating Response**

#Prompt 1 - Simple Colon

In [ ]:
file_path = f'/content/drive/MyDrive/IMDB_Simple_Prompt1.csv'

In [ ]:
result_df = pd.DataFrame()

for i, row in tqdm(df.iterrows()):

  review_text = df['text'][i]

  prompt_template=f'''Review:{review_text}
  Sentiment(Positive/Negative):
  '''

  response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

  input_text = response["choices"][0]["text"]

  output = input_text.split("Sentiment(Positive/Negative):\n")[1].strip()
  print(i, output)

  row = {'Review':review_text, 'Result':output}

  result_df = result_df.append(row, ignore_index=True)

  with open(file_path, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(list(row.values()))

result_df.to_csv('result_df.csv', index=False)

0it [00:00, ?it/s]Llama.generate: prefix-match hit
<ipython-input-28-6492b2b60d41>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
1it [00:57, 57.89s/it]

69 Negative
Review:I loved the movie, but I don't agree that it is trashy or sends a bad message. It is a lighthearted comedy and Steve Martin's performance was great as always. The scene with Charlene encouraging him to be rough with his wife may have been inappropriate for some viewers, but overall the movie was funny and entertaining.


Llama.generate: prefix-match hit
<ipython-input-28-6492b2b60d41>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
2it [01:10, 31.18s/it]

70 Positive


Llama.generate: prefix-match hit
<ipython-input-28-6492b2b60d41>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
3it [02:29, 52.94s/it]

...

Llama.generate: prefix-match hit
<ipython-input-28-6492b2b60d41>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
684it [5:10:07, 35.18s/it]

752 Positive: 0%
   Negative: 100%
  Recommendation:No recommendation (do not watch).


Llama.generate: prefix-match hit
<ipython-input-28-6492b2b60d41>:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
685it [5:10:16, 27.47s/it]

753 Positive


Llama.generate: prefix-match hit


#Prompt 2 - Master

In [ ]:
file_path = f'/content/drive/MyDrive/IMDB_Master_Prompt2.csv'

In [ ]:
result_df = pd.DataFrame()

for i, row in df.iterrows():

  review_text = df['text'][i]

  prompt_template=f'''A review is provided:{review_text}
  Based on the sentiment of review, the label can be either 0 or 1. In case there is any other category, try to categorize it in either 0 or 1.
  The masterful movie review Sentiment Analyzer gives it a binary label:
  '''

  response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

  input_text = response["choices"][0]["text"]

  output = input_text.split("The masterful movie review Sentiment Analyzer gives it a binary label:\n")[1].strip()
  print(output)

  row = {'Review':review_text, 'Result':output}

  result_df = result_df.append(row, ignore_index=True)

  with open(file_path, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(list(row.values()))

result_df.to_csv('result_df.csv', index=False)

Llama.generate: prefix-match hit


1 - Positive


<ipython-input-35-7e51031fbae9>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
Llama.generate: prefix-match hit


0


<ipython-input-35-7e51031fbae9>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
Llama.generate: prefix-match hit


...

<ipython-input-35-7e51031fbae9>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
Llama.generate: prefix-match hit


Label: 1 (positive)


<ipython-input-35-7e51031fbae9>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
Llama.generate: prefix-match hit


1 - Very Negative


<ipython-input-35-7e51031fbae9>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)


#Prompt 3 - Master Enhanced

In [ ]:
file_path = f'/content/drive/MyDrive/IMDB-Master_Prompt3.csv'

In [ ]:
result_df = pd.DataFrame()

for i, row in tqdm(df.iterrows()):

  review_text = df['text'][i]

  prompt_template=f'''A review is provided:{review_text}
  Based on the sentiment of review, the label can be either 0 or 1. In case there is any other category, try to categorize it in either 0 or 1.
  The extremely efficient masterful movie review Sentiment Analyzer gives it a binary label:
  '''

  response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

  input_text = response["choices"][0]["text"]

  output = input_text.split("The extremely efficient masterful movie review Sentiment Analyzer gives it a binary label:\n")[1].strip()
  # print(output)

  row = {'Review':review_text, 'Result':output}

  result_df = result_df.append(row, ignore_index=True)

  with open(file_path, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(list(row.values()))

result_df.to_csv('result_df.csv', index=False)

0it [00:00, ?it/s]Llama.generate: prefix-match hit
<ipython-input-17-cc8d04e9c576>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
1it [00:06,  6.18s/it]Llama.generate: prefix-match hit
<ipython-input-17-cc8d04e9c576>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
2it [00:15,  8.19s/it]Llama.generate: prefix-match hit
<ipython-input-17-cc8d04e9c576>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
3it [00:27,  9.89s/it]Llama.generate: prefix-match hit
<ipython-input-17-cc8d04e9c576>:23: FutureWarning: The frame.append method is deprecated and will be removed fro

KeyboardInterrupt: ignored

#Prompt 4 - Master Discouraged

In [ ]:
file_path = f'/content/drive/MyDrive/IMDB-Master_Prompt4.csv'

In [ ]:
result_df = pd.DataFrame()

for i, row in tqdm(df.iterrows()):

  review_text = df['text'][i]

  prompt_template=f'''A review is provided:{review_text}
  Based on the sentiment of review, the label can be either 0 or 1. In case there is any other category, try to categorize it in either 0 or 1.
  The incompetent masterful movie review Sentiment Analyzer gives it a binary label:
  '''

  response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

  input_text = response["choices"][0]["text"]

  output = input_text.split("The incompetent masterful movie review Sentiment Analyzer gives it a binary label:\n")[1].strip()
  # print(output)

  row = {'Review':review_text, 'Result':output}

  result_df = result_df.append(row, ignore_index=True)

  with open(file_path, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(list(row.values()))

result_df.to_csv('result_df.csv', index=False)

0it [00:00, ?it/s]Llama.generate: prefix-match hit
<ipython-input-17-994f319b8566>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
1it [00:01,  1.83s/it]Llama.generate: prefix-match hit
<ipython-input-17-994f319b8566>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
2it [00:07,  4.29s/it]Llama.generate: prefix-match hit
<ipython-input-17-994f319b8566>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
3it [00:15,  6.00s/it]Llama.generate: prefix-match hit
<ipython-input-17-994f319b8566>:23: FutureWarning: The frame.append method is deprecated and will be removed fro

#Prompt 5 - Few Shot Prompt (1-Shot Simple Colon)

In [ ]:
file_path = f'/content/drive/MyDrive/IMDB-FewShot_Prompt5.csv'

In [ ]:
data_one_shot=dataset_imdb[1000:].sample(n=1000, random_state=1).reset_index()

In [ ]:
text_1shot=data_one_shot['text'][2]
label_1shot=data_one_shot['label'][2]

In [ ]:
text_1shot

"This film has absolutely no redeeming features. It's not even worthy of being in the 'so bad it's good' category - it's simply bad. Badly acted, badly shot, badly written, badly directed, terrible sound recording and the whole thing is just incredibly amateur. Quite how this got a release is beyond me.<br /><br />Oh, and could someone PLEASE tell the lead actress that if you're playing 'mad' that does not mean that you should constantly fiddle with your hair - in a film that was generally irritating, this was probably one of the most irritating things of all.<br /><br />On the plus side, I think this is the only film ever to feature a Nazi carjacker ..."

In [ ]:
result_df = pd.DataFrame()
for i, row in tqdm(df.iterrows()):

  review_text = df['text'][i]
  print(review_text)

  prompt_template=f'''Review:{text_1shot}
  Sentiment(Positive/Negative):{label_1shot}
  Review:{review_text}
  Sentiment(Positive/Negative):
  '''

  response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

  input_text = response["choices"][0]["text"]
  output = input_text.split("Sentiment(Positive/Negative):\n")[-1].strip()
  row = {'Review':review_text, 'Result':output}

  result_df = result_df.append(row, ignore_index=True)

  with open(file_path, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(list(row.values()))

result_df.to_csv('result_df.csv', index=False)

#Prompt 6 - Few Shot Prompt (3-Shot Simple Colon)

In [ ]:
file_path = f'/content/drive/MyDrive/IMDB-FewShot_Prompt6.csv'

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/IMDB-FewShot_Prompt5_Subset-N_A-50.csv")

In [ ]:
df

,Review,Output,Final
0,"Man, is it great just to see Young and The Res...",Review:This is one of those movies that you wa...,NaN
1,Anyone who appreciates fine acting and ringing...,NaN,NaN
2,This is one of Julie's greatest tributes to mu...,Review:This movie has no redeeming qualities. ...,NaN
3,"Okay... she's on the boat with this guy, reali...",Review:I can't believe how much money this fil...,NaN
4,"It's just when a band tours, and only has one ...",Review:This is such a terrible film that I can...,NaN
5,This movie is simply awesome. It is so hilario...,Review:This movie was a complete waste of time...,NaN
6,Project A II is a classic Jackie Chan movie wi...,Review:This film has absolutely no redeeming f...,NaN
7,"This is not a serious film, and does not prete...",Review:This film was so bad that I can't even ...,NaN
8,The fun that was present in the other 'movies'...,Review:The first two episodes were fun but thi...,NaN
9,"quite good, don't expect anything high culture...",Review:Bad acting and poor production value bu...,NaN


In [ ]:
result_df = pd.DataFrame()

for i, row in tqdm(df.iloc[0:50].iterrows()):

  review_text = df['Review'][i]

  prompt_template=f'''Review:its a totally average film with a few semi-alright action sequences that make the plot seem a little better and remind the viewer of the classic van dam films. parts of the plot don't make sense and seem to be added in to use up time. the end plot is that of a very basic type that doesn't leave the viewer guessing and any twists are obvious from the beginning. the end scene with the flask backs don't make sense as they are added in and seem to have little relevance to the history of van dam's character. not really worth watching again, bit disappointed in the end production, even though it is apparent it was shot on a low budget certain shots and sections in the film are of poor directed quality
  Sentiment(Positive/Negative): 0
  Review:The Forgotten (AKA: Don't Look In The Basement) is a very cheaply made and very old looking horror movie.<br /><br />The story is very slow and never really reaches anything worth getting excited about.<br /><br />The patients at the asylum are embarrassingly funny especially Sam and the old woman who always quotes an old saying to everyone. (Look out for the bit when she gets close to the camera, tell me you can watch without laughing!).<br /><br />Now the gore is very poor looking, with the blood looking pink in many scenes so it doesn't really deserve its place on the video nasties list!.<br /><br />Overall if you aren't looking for a fantastic horror film and have some time to spare then it's worth a watch.
  Sentiment(Positive/Negative): 0
  Review:There won't be one moment in this film where you aren't laughing. This is Mel Brooks at one of his high points, and Cary Elwes carries off the part of Robin with convincing humour. Every time you watch this film you will discover a new joke, but the ones you have noticed before will never grow old. Highly recomended!
  Sentiment(Positive/Negative): 1
  Review:{review_text}
  Sentiment(Positive/Negative):
  '''

  response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

  input_text = response["choices"][0]["text"]

  output = input_text.split("Sentiment(Positive/Negative):\n")[-1].strip()

  row = {'Review':review_text, 'Result':output}

  result_df = result_df.append(row, ignore_index=True)

  with open(file_path, 'a', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(list(row.values()))

result_df.to_csv('result_df.csv', index=False)

0it [00:00, ?it/s]Llama.generate: prefix-match hit
<ipython-input-33-6d2ed0eb1bff>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
1it [00:00,  1.06it/s]Llama.generate: prefix-match hit
<ipython-input-33-6d2ed0eb1bff>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
2it [01:08, 39.85s/it]Llama.generate: prefix-match hit
<ipython-input-33-6d2ed0eb1bff>:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(row, ignore_index=True)
3it [01:12, 23.53s/it]Llama.generate: prefix-match hit
<ipython-input-33-6d2ed0eb1bff>:27: FutureWarning: The frame.append method is deprecated and will be removed fro